## Big Query MPE back office

### Environment settings

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import gspread
import duckdb
# Import authenticator and gspread to manage g-sheets
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from google.cloud import bigquery
import connectorx as cx

In [2]:
# Create sheets and drive scopes to authenticate
scopes = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/analytics.readonly']

# Read google credentials
api = '../APIS/gepp-538-db.json'
# connect to google sheets
gs_credentials = ServiceAccountCredentials.from_json_keyfile_name(api, scopes)
gc = gspread.authorize(gs_credentials)
# connect to big query
bq_credentials = service_account.Credentials.from_service_account_file(api)
project_id = 'gepp-538'
client = bigquery.Client(credentials=bq_credentials,project=project_id)

### Duckdb

In [27]:
# consulta detallada usuarios
mpe_users = pl.read_csv('../usuarios.csv', separator=';', ignore_errors=True)

In [28]:
# create sql queries with duckdb
# read file with duckdb (csv, parquet, json)
duckdb.sql(
    '''
    SELECT
        Origen, COUNT(Nombre) AS usuarios
    FROM '../usuarios.csv'
    GROUP BY Origen
    ORDER BY usuarios DESC
    '''
).pl() # and pipeline to polars dataframe
#others formats include: .pdf() for pandas, .arrow() for arrow, and .fetchnumpy() for numpy arrays

Origen,usuarios
str,i64
"""WA""",765173
"""Ecommerce""",97913


**create files from duckdb queries**
* duckdb.sql('SELECT 42').write_parquet('out.parquet') # Write to a Parquet file
* duckdb.sql('SELECT 42').write_csv('out.csv')         # Write to a CSV file
* duckdb.sql("COPY (SELECT 42) TO 'out.parquet'")      # Copy to a parquet file

### MPE

In [113]:
mpe_hist = (
    pl.read_csv('../MPE_gral.csv', infer_schema_length=0, ignore_errors=True)
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename({
    'fecha_alta':'fecha_de_pedido',
    'fecha_modificaci�n':'fecha_modificacion',
    'hora_modificaci�n':'hora_modificacion',
    'tel�fono_de_contacto':'tel_contacto',
    'alias_de_direcci�n':'alias_dir',
    'direcci�n_completa':'dir_completa',
    'num._mes':'num_ mes',
    'a�o':'year',
    'mes|a�o':'month',
    'id|_cedis':'id_cedis',
    'regi�n':'region'
    }).with_columns(
        pl.col(['fecha_de_pedido','fecha_modificacion','fecha_entrega']
              ).str.strptime(pl.Datetime, strict=False))
    .select(pl.exclude('num_ mes','mes','year','month','id_cedis','region'))
)

In [96]:
mpe_mzo = (
    pl.read_excel('../mpe-pedidos 2023-03-31.xlsx', read_csv_options={'infer_schema_length':0},)
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'fecha_modificación':'fecha_modificacion',
        'hora_modificación':'hora_modificacion',
        'teléfono_de_contacto':'tel_contacto',
        'alias_de_dirección':'alias_dir',
        'dirección_completa':'dir_completa',
    }
    ).with_columns(
        pl.col(['fecha_de_pedido','fecha_modificacion','fecha_entrega']
              ).str.strptime(pl.Datetime, strict=False))
)

In [97]:
mpe_abr = (
    pl.read_excel('../mpe-pedidos 2023-04-30.xlsx', read_csv_options={'infer_schema_length':0},)
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'fecha_modificación':'fecha_modificacion',
        'hora_modificación':'hora_modificacion',
        'teléfono_de_contacto':'tel_contacto',
        'alias_de_dirección':'alias_dir',
        'dirección_completa':'dir_completa',
    }
    ).with_columns(
        pl.col(['fecha_de_pedido','fecha_modificacion','fecha_entrega']
              ).str.strptime(pl.Datetime, strict=False))
)

In [98]:
mpe_may1 = (
    pl.read_excel('../mpe-pedidos 2023-05-15.xlsx', read_csv_options={'infer_schema_length':0},)
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'fecha_modificación':'fecha_modificacion',
        'hora_modificación':'hora_modificacion',
        'teléfono_de_contacto':'tel_contacto',
        'alias_de_dirección':'alias_dir',
        'dirección_completa':'dir_completa',
    }
    ).with_columns(
        pl.col(['fecha_de_pedido','fecha_modificacion','fecha_entrega']
              ).str.strptime(pl.Datetime, strict=False))
)

In [99]:
mpe_may2 = (
    pl.read_excel('../mpe-pedidos 2023-05-23.xlsx', read_csv_options={'infer_schema_length':0},)
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'fecha_modificación':'fecha_modificacion',
        'hora_modificación':'hora_modificacion',
        'teléfono_de_contacto':'tel_contacto',
        'alias_de_dirección':'alias_dir',
        'dirección_completa':'dir_completa',
    }
    ).with_columns(
        pl.col(['fecha_de_pedido','fecha_modificacion','fecha_entrega']
              ).str.strptime(pl.Datetime, strict=False))
)

In [102]:
mpe_pedidos = (
    pl.concat([mpe_hist, mpe_mzo, mpe_abr, mpe_may1, mpe_may2])
    .with_columns(
        [
        pl.col('cantidad_de_productos').cast(pl.Int64, strict=False),
        pl.col('total_de_orden').cast(pl.Float64, strict=False),
        pl.col('promociones').cast(pl.Int64, strict=False),
        pl.col('ruta_de_entrega').cast(pl.Int64, strict=False),
        pl.col('id_bodega').cast(pl.Int64, strict=False),
        pl.col('nud').cast(pl.Int64, strict=False),
        ]
    )
)

In [103]:
mpe_bodegas = pl.read_csv('../mpe-catalogo-bodegas.csv')

In [104]:
mpe_pedidos = (
    mpe_pedidos.join(mpe_bodegas, on='id_bodega', how='left').select(
        pl.exclude(
            ['clave_bodega_destino','tipo_preventa_tradicional','clave_bodega_origen']
        )
    )
)

In [21]:
mpe_det_hist = (
    pl.read_excel('../MPE_Reporte_2023.xlsx', sheet_name='Detalle', read_csv_options={'infer_schema_length':0})
).select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
)

In [46]:
mpe_det_hist = mpe_det_hist.rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )

In [40]:
mpe_det_mzo1 = (
    pl.read_excel(
        '../mpe-detalle-2023-03-15.xlsx',
        sheet_name='Pedidos Detalle',
        read_csv_options={'infer_schema_length':0})
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
    ).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )
)

In [41]:
mpe_det_mzo2 = (
    pl.read_excel(
        '../mpe-detalle-2023-03-31.xlsx',
        sheet_name='Pedidos Detalle',
        read_csv_options={'infer_schema_length':0})
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
    ).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )
)

In [42]:
mpe_det_abr1 = (
    pl.read_excel(
        '../mpe-detalle-2023-04-15.xlsx',
        sheet_name='Pedidos Detalle',
        read_csv_options={'infer_schema_length':0})
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
    ).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )
)

In [43]:
mpe_det_abr2 = (
    pl.read_excel(
        '../mpe-detalle-2023-04-30.xlsx',
        sheet_name='Pedidos Detalle',
        read_csv_options={'infer_schema_length':0})
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
    ).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )
)

In [44]:
mpe_det_may1 = (
    pl.read_excel(
        '../mpe-detalle-2023-05-15.xlsx',
        sheet_name='Pedidos Detalle',
        read_csv_options={'infer_schema_length':0})
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
    ).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )
)

In [45]:
mpe_det_may2 = (
    pl.read_excel(
        '../mpe-detalle-2023-05-23.xlsx',
        sheet_name='Pedidos Detalle',
        read_csv_options={'infer_schema_length':0})
    .select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'producto(s)':'productos'
    }
    ).with_columns(
    pl.col('cantidad').cast(pl.Int64),
    pl.col('precio_por_unidad').cast(pl.Float64),
    pl.col('descuento_por_unidad').cast(pl.Float64),
    pl.col('precio_acumulado').cast(pl.Float64),
    pl.col('descuento_acumulado').cast(pl.Float64),
    )
)

In [50]:
mpe_detalle = pl.concat(
    [mpe_det_hist, mpe_det_mzo1, mpe_det_mzo2, mpe_det_abr1, mpe_det_abr2, mpe_det_may1, mpe_det_may2]
)

In [64]:
(
    pl.read_excel('../MPE_Reporte_2023.xlsx', sheet_name='Usuarios')
    .select(pl.exclude('Num. Mes','Mes','Año','Mes | Año'))
    .write_csv('users.csv')
)

In [72]:
mpe_usuarios = pl.read_csv('~/iCloud/mpe-usuarios.csv').with_columns(pl.col('fecha').str.strptime(pl.Datetime))

### GETM

In [59]:
# Pedidos General
# Pedidos Detalle
getm1 = pl.read_excel('~/iCloud/getm 2021-06.xlsx', sheet_name='Pedidos Detalle', 
                      read_csv_options={'infer_schema_length':0})

In [60]:
getm2 = pl.read_excel('~/iCloud/getm 2021-12.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [61]:
getm3 = pl.read_excel('~/iCloud/getm 2022-03.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [62]:
getm4 = pl.read_excel('~/iCloud/getm 2022-06.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [63]:
getm5 = pl.read_excel('~/iCloud/getm 2022-07.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [64]:
getm6 = pl.read_excel('~/iCloud/getm 2022-08.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [65]:
getm7 = pl.read_excel('~/iCloud/getm 2022-09.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [66]:
getm8 = pl.read_excel('~/iCloud/getm 2022-12.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [67]:
getm9 = pl.read_excel('~/iCloud/getm 2023-01.xlsx', sheet_name='Pedidos Detalle',
                     read_csv_options={'infer_schema_length':0})

In [68]:
getm10 = pl.read_excel('~/iCloud/getm 2023-02.xlsx', sheet_name='Pedidos Detalle',
                      read_csv_options={'infer_schema_length':0})

In [69]:
getm11 = pl.read_excel('~/iCloud/getm 2023-03.xlsx', sheet_name='Pedidos Detalle',
                      read_csv_options={'infer_schema_length':0})

In [70]:
getm12 = pl.read_excel('~/iCloud/getm 2023-04.xlsx', sheet_name='Pedidos Detalle',
                      read_csv_options={'infer_schema_length':0})

In [71]:
getm13 = pl.read_excel('~/iCloud/getm 2023-05-23.xlsx', sheet_name='Pedidos Detalle',
                      read_csv_options={'infer_schema_length':0})

In [72]:
getm = pl.concat([getm1,getm2,getm3,getm4,getm5,getm6,getm7,getm8,getm9,getm10,getm11,getm12,getm13,])

In [80]:
getm_detalle = (
    getm.select(
        pl.col('*').map_alias(lambda col_name: col_name.lower().replace(' ', '_'))
    ).rename(
    {
        'clave_de_promoción':'clave_de_promocion',
        'tipo_de_promoción':'tipo_de_promocion',
        'descripción_de_promoción':'descripcion_de_promocion',
        'producto(s)':'productos',
    }
    ).with_columns(
        pl.col(['id_bodega','nud']).cast(pl.Int64),
        pl.col(['cantidad','precio_por_unidad','descuento_por_unidad','precio_acumulado','descuento_acumulado'
            ]).cast(pl.Float64))
)

### Upload to Big Query

### MPE

In [72]:
# create dataset gbq api
#client.create_dataset('back_office')

Dataset(DatasetReference('gepp-538', 'back_office'))

In [107]:
mpe_pedidos = mpe_pedidos.to_pandas()
mpe_pedidos.to_gbq('gepp-538.back_office.mpe_pedidos',
                    project_id='gepp-538',
                    if_exists='replace',
                    credentials=bq_credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 6061.13it/s]


In [52]:
mpe_detalle = mpe_detalle.to_pandas()
mpe_detalle.to_gbq('gepp-538.back_office.mpe_detalle',
                    project_id='gepp-538',
                    if_exists='replace',
                    credentials=bq_credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 8256.50it/s]


In [73]:
mpe_usuarios = mpe_usuarios.to_pandas()
mpe_usuarios.to_gbq('gepp-538.back_office.mpe_usuarios',
                    project_id='gepp-538',
                    if_exists='replace',
                    credentials=bq_credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3030.57it/s]


### GETM

In [41]:
getm_pedidos = getm_pedidos.to_pandas()
getm_pedidos.to_gbq('gepp-538.back_office.getm_pedidos',
                    project_id='gepp-538',
                    if_exists='replace',
                    credentials=bq_credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2576.35it/s]


In [81]:
getm_detalle = getm_detalle.to_pandas()
getm_detalle.to_gbq('gepp-538.back_office.getm_detalle',
                    project_id='gepp-538',
                    if_exists='replace',
                    credentials=bq_credentials)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3460.65it/s]


### Retrieve datasets from Big Query

In [ ]:
# create sql query
query = '''
    SELECT * FROM `gepp-538.transformation.mpe_catalogo`
'''
# convert query to pandas dataframe
mpe_catalogo = pd.read_gbq(query, credentials=bq_credentials)